## Создание сводных таблиц. Агрегатные функции.

In [39]:
import pandas as pd
import numpy as np
import sys
import warnings
if not sys.warnoptions:
       warnings.simplefilter("ignore")

pandas**.pivot_table**(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False, sort=True)

*data* - датафрейм;

*values* - столбцы для агрегации (опционально);

*index* - ключи для группировки по индексу;

*columns* - ключи для групировки столбцов;

*aggfunc* - функция используемая для агрегации значений;

*fill_values* - Значение для замены отсутствующих значений (в результирующей сводной таблице после агрегирования);

*dropna* - включать/не включать столбцы, где все значения NaN;

*sort* - наличие/отсутствие сортировки.

In [40]:
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                         "bar", "bar", "bar", "bar"],
                   "B": ["one", "one", "one", "two", "two",
                         "one", "one", "two", "two"],
                   "C": ["small", "large", "large", "small",
                         "small", "large", "small", "small",
                         "large"],
                   "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})

In [41]:
df

,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,one,large,2,5
3,foo,two,small,3,5
4,foo,two,small,3,6
5,bar,one,large,4,6
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


In [42]:
table = pd.pivot_table(df, values='D', index=['A', 'B'],
                    columns=['C'], aggfunc=np.sum)
table

C        large  small
A   B                
bar one    4.0    5.0
    two    7.0    6.0
foo one    4.0    1.0
    two    NaN    6.0

Воспользуемся опцией fill_value для заполнения пропусков в сводной таблице.

In [43]:
table = pd.pivot_table(df, values='D', index=['A', 'B'],
                    columns=['C'], aggfunc=np.sum, fill_value=0)
table

C        large  small
A   B                
bar one      4      5
    two      7      6
foo one      4      1
    two      0      6

Рассчитаем среднее значение по двум столбцам.

In [44]:
table = pd.pivot_table(df, values=['D', 'E'], index=['A', 'C'],
                    aggfunc={'D': np.mean,
                             'E': np.mean})
table

D         E
A   C                        
bar large  5.500000  7.500000
    small  5.500000  8.500000
foo large  2.000000  4.500000
    small  2.333333  4.333333

Применим к разным столбцам разным функции агрегации.

In [45]:
table = pd.pivot_table(df, values=['D', 'E'], index=['A', 'C'],
                    aggfunc={'D': np.mean,
                             'E': [min, max, np.mean]})
table

D    E               
               mean  max      mean  min
A   C                                  
bar large  5.500000  9.0  7.500000  6.0
    small  5.500000  9.0  8.500000  8.0
foo large  2.000000  5.0  4.500000  4.0
    small  2.333333  6.0  4.333333  2.0

Загружаем данные из .csv файла

In [46]:
data = pd.read_csv('Online Retail.csv', sep = ';', parse_dates = ['InvoiceDate'])  

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Проверяем как загрузились данные (вывод первых, последних и слуайно выбранных строк).

In [47]:
data.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Unnamed: 8
0,581483,23843,PAPER CRAFT . LITTLE BIRDIE,80995,2011-09-12 09:15:00,2.08,16446.0,United Kingdom,NaN


In [48]:
df.tail(3)

,A,B,C,D,E
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


In [49]:
df.sample(3, random_state = 123)

,A,B,C,D,E
7,bar,two,small,6,9
0,foo,one,small,1,2
5,bar,one,large,4,6


Удаляем "лишний" столбец.

In [50]:
df = data.drop(columns = ['Unnamed: 8'])

In [51]:
df['Quantity'] = df['Quantity'].astype('int')
df['CustomerID'] = df['CustomerID'].astype('str')
df['UnitPrice'] = df['UnitPrice'].astype('float')
df['Country'] = df['Country'].astype('category')

In [52]:
df.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,581483,23843,PAPER CRAFT . LITTLE BIRDIE,80995,2011-09-12 09:15:00,2.08,16446.0,United Kingdom


In [53]:
df['revenue'] = df['Quantity']*df['UnitPrice']

In [54]:
df.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
0,581483,23843,PAPER CRAFT . LITTLE BIRDIE,80995,2011-09-12 09:15:00,2.08,16446.0,United Kingdom,168469.6


In [68]:
table = pd.pivot_table(df, index=['Country'],
               #columns=['revenue', 'Quantity'],
               aggfunc={'revenue': [np.mean, sum],
                        'Quantity': np.mean})
table

Quantity     revenue             
                           mean        mean          sum
Country                                                 
Australia             66.444003  108.877895   137077.270
Austria               12.037406   25.322494    10154.320
Bahrain               13.684211   28.863158      548.400
Belgium               11.189947   19.773301    40910.960
Brazil                11.125000   35.737500     1143.600
Canada                18.298013   24.280662     3666.380
Channel Islands       12.505277   26.499063    20086.290
Cyprus                10.155949   20.813971    12946.290
Czech Republic        19.733333   23.590667      707.720
Denmark               21.048843   48.247147    18768.140
EIRE                  17.403245   32.122599   263276.820
European Community     8.147541   21.176230     1291.750
Finland               15.346763   32.124806    22326.740
France                12.911067   23.069288   197403.900
Germany               12.369458   23.348943   221698.210
Greece                10.657534   32.263836     4710.520
Hong Kong             16.559028   35.128611    10117.040
Iceland               13.505495   23.681319     4310.000
Israel                14.656566   26.625657     7907.820
Italy                  9.961395   21.034259    16890.510
Japan                 70.441341   98.716816    35340.620
Lebanon                8.577778   37.641778     1693.880
Lithuania             18.628571   47.458857     1661.060
Malta                  7.433071   19.728110     2505.470
Netherlands           84.406580  120.059696   284661.540
Norway                17.722836   32.378877    35163.460
Poland                10.712610   21.152903     7213.140
Portugal              10.651745   19.333127    29367.020
RSA                    6.068966   17.281207     1002.310
Saudi Arabia           7.500000   13.117000      131.170
Singapore             22.855895   39.827031     9120.390
Spain                 10.589814   21.624390    54774.580
Sweden                77.136364   79.211926    36595.910
Switzerland           15.147353   28.164510    56385.350
USA                    3.553265    5.948179     1730.920
United Arab Emirates  14.441176   27.974706     1902.280
United Kingdom         8.605486   16.525065  8187806.364
Unspecified            7.399103   10.649753     4749.790

In [56]:
df['Country'].value_counts()

United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

In [57]:
df_ = df[(df['Country'] == 'Germany') | (df['Country'] == 'Japan') | (df['Country'] == 'Austria') | (df['Country'] == 'Australia') |(df['Country'] == 'Singapore')]
len(df_)

11742

In [58]:
df_1 = df[df['Country'].isin(['Germany', 'Japan', 'Austria', 'Australia', 'Singapore'])]
len(df_1)

11742

In [59]:
df_short = df[df['Country'].isin(['Japan', 'Austria', 'Singapore'])]
len(df_short)

988

In [60]:
df_1['InvoiceMonth'] = df_1['InvoiceDate'].dt.month

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [61]:
df_final = df_1[df_1['InvoiceMonth'] <= 3]
len(df_final)

2193

In [62]:
df_final.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue,InvoiceMonth
99,565145,22492,MINI PAINT SET VINTAGE,1152,2011-01-09 13:50:00,0.55,12415.0,Australia,633.6,1


In [63]:
df_final = df_final[['Country','Quantity','InvoiceNo','CustomerID','revenue','InvoiceDate']]

In [64]:
table = pd.pivot_table(df_short, index=['Country'],
               #columns=['revenue', 'Quantity'],
               aggfunc={'revenue': sum,
                        'Quantity': np.mean})

In [65]:
table

,Quantity,revenue
Country,,
Australia,NaN,0.00
Austria,12.037406,10154.32
Bahrain,NaN,0.00
Belgium,NaN,0.00
Brazil,NaN,0.00
Canada,NaN,0.00
Channel Islands,NaN,0.00
Cyprus,NaN,0.00
Czech Republic,NaN,0.00
